# メンターの方が教えてくれたベースのやつ

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import datetime
import unicodedata
import numpy as np

In [ ]:
data = pd.read_csv("train.csv")
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
submit = pd.read_csv("submit_sample.csv", header=None)

In [ ]:
#　表記ゆれ修正


# Convert all values to lowercase and replace full-width characters with half-width characters
data['manufacturer'] = data['manufacturer'].str.lower().apply(lambda x: unicodedata.normalize('NFKC', x)) #データフレームdataのmanufacturer列のすべての文字を小文字に変換します。
#unicodedata.normalize('NFKC', x)関数を使用して、manufacturer列のすべての文字を半角に変換します。これにより、全角のアルファベット文字が半角に変換されます。

# Manually fix specific character inconsistencies
data['manufacturer'] = data['manufacturer'].replace({ #replace関数を使用して、manufacturer列の特定の文字列を他の文字列に置き換えます。このコードでは、特定のメーカー名において文字が誤って入力されているケースがあるため、これらの誤りを修正しています。
    'niѕsan': 'nissan',
    'nisѕan': 'nissan',
    'subαru': 'subaru',
    'sαturn': 'saturn',
    'аcura': 'acura',
    'vоlkswagen': 'volkswagen',
    'lexuѕ': 'lexus',
    'ᴄhrysler': 'chrysler'
})
#このコードの目的は、データの整合性を保ち、後の分析での問題を避けるために、manufacturer列のテキストデータをクレンジングすることです。



In [ ]:
df_test['manufacturer'] = df_test['manufacturer'].str.lower().apply(lambda x: unicodedata.normalize('NFKC', x)) #データフレームdataのmanufacturer列のすべての文字を小文字に変換します。
#unicodedata.normalize('NFKC', x)関数を使用して、manufacturer列のすべての文字を半角に変換します。これにより、全角のアルファベット文字が半角に変換されます。

# Manually fix specific character inconsistencies
df_test['manufacturer'] = df_test['manufacturer'].replace({ #replace関数を使用して、manufacturer列の特定の文字列を他の文字列に置き換えます。このコードでは、特定のメーカー名において文字が誤って入力されているケースがあるため、これらの誤りを修正しています。
    'niѕsan': 'nissan',
    'nisѕan': 'nissan',
    'subαru': 'subaru',
    'sαturn': 'saturn',
    'аcura': 'acura',
    'vоlkswagen': 'volkswagen',
    'lexuѕ': 'lexus',
    'ᴄhrysler': 'chrysler'
})

In [ ]:
size_corrections = { #size_correctionsという名前のディクショナリを定義します。このディクショナリのキーは、データフレームのsize列で置き換えたい文字列です。対応する値は、置き換え後の文字列です。
    "full−size": "full-size",
    "fullーsize": "full-size",
    "mid−size": "mid-size",
    "midーsize": "mid-size",
    "subーcompact": "sub-compact"
}

data['size'] = data['size'].replace(size_corrections) #replace関数を使用して、size列の文字列をsize_correctionsディクショナリに基づいて置き換えます。


In [ ]:
size_corrections = { #size_correctionsという名前のディクショナリを定義します。このディクショナリのキーは、データフレームのsize列で置き換えたい文字列です。対応する値は、置き換え後の文字列です。
    "full−size": "full-size",
    "fullーsize": "full-size",
    "mid−size": "mid-size",
    "midーsize": "mid-size",
    "subーcompact": "sub-compact"
}

df_test['size'] = df_test['size'].replace(size_corrections) #replace関数を使用して、size列の文字列をsize_correctionsディクショナリに基づいて置き換えます。


In [ ]:
# yearの値修正


current_year = datetime.datetime.now().year #datetimeモジュールを使って、現在の年をcurrent_yearという変数に格納します。

# Identify years that are greater than current year + 1
anomalous_years = data[data['year'] > current_year + 1]['year'].unique() #dataデータフレームのyear列の値が現在の年よりも大きい（現在の年+1以上の値）場合、それらの値をanomalous_yearsという変数に保存します。

# Correct the anomalous years by subtracting 1000
data['year'] = data['year'].replace({year: year - 1000 for year in anomalous_years}) #replace関数を使用して、year列の中でanomalous_yearsに含まれる値を、その値から1000を引いた値に置き換えます


#このコードの目的は、year列の不適切な値を修正して、データの品質を向上させることです。これにより、後の分析での問題が回避されます。



In [ ]:
current_year = datetime.datetime.now().year #datetimeモジュールを使って、現在の年をcurrent_yearという変数に格納します。

# Identify years that are greater than current year + 1
anomalous_years = df_test[df_test['year'] > current_year + 1]['year'].unique() #dataデータフレームのyear列の値が現在の年よりも大きい（現在の年+1以上の値）場合、それらの値をanomalous_yearsという変数に保存します。

# Correct the anomalous years by subtracting 1000
df_test['year'] = df_test['year'].replace({year: year - 1000 for year in anomalous_years}) #replace関数を使用して、year列の中でanomalous_yearsに含まれる値を、その値から1000を引いた値に置き換えます



In [ ]:
# 走行距離の分類と車製造年の特徴量追加→精度悪化したので保留

# quantiles = data['odometer'].quantile([0.25, 0.5, 0.75])
# conditions = [
#     data['odometer'] <= quantiles[0.25],
#     (data['odometer'] > quantiles[0.25]) & (data['odometer'] <= quantiles[0.5]),
#     data['odometer'] > quantiles[0.5]
# ]
# choices = ['low', 'medium', 'high']

# data['odometer_usage'] = np.select(conditions, choices, default='unknown')

# # 2. Create a new feature for car age
# data['car_age'] = current_year - data['year']

# # Check the first few rows of the dataframe to confirm the changes
# data[['odometer', 'odometer_usage', 'year', 'car_age']].head()

In [ ]:
from sklearn.model_selection import train_test_split

# Filling missing values
#欠損値の補完: fuel、title_status、type、およびstateという列について、欠損値をそれぞれの列の最頻値で補完しています。
data['fuel'].fillna(data['fuel'].mode()[0], inplace=True)
data['title_status'].fillna(data['title_status'].mode()[0], inplace=True)
data['type'].fillna(data['type'].mode()[0], inplace=True)
data['state'].fillna(data['state'].mode()[0], inplace=True)

# Encoding categorical columns
#カテゴリ変数のエンコード: データフレームのカテゴリ変数（文字列などの非数値型の列）を数値に変換しています。これは、機械学習モデルは数値データしか受け付けないためです。カテゴリ変数の列を選択し、各列をエンコードして数値に変換しています。
categorical_cols = data.select_dtypes(include=['object']).columns
for col in categorical_cols:
    data[col] = data[col].astype('category').cat.codes

# Splitting the data into train and validation sets (80:20)
#データの分割: train_test_split関数を使用して、データを訓練セットと検証セットに分割しています。分割比率は80:20です。random_state引数に42を設定することで、結果が再現可能になります。
train_data, valid_data = train_test_split(data, test_size=0.2, random_state=42)

# Separating the target variable
#ターゲット変数の分離: ターゲット変数（予測対象の変数）であるprice列と、一意のIDであるid列を除いた残りの列を特徴量として使用します。これらのデータをX_train、y_train、X_valid、およびy_validという変数に格納しています。
X_train = train_data.drop(columns=['price', 'id'])
y_train = train_data['price']
X_valid = valid_data.drop(columns=['price', 'id'])
y_valid = valid_data['price']

#最後に、訓練セットと検証セットの形状を表示しています。
X_train.shape, X_valid.shape

((22025, 14), (5507, 14))

In [ ]:
# Filling missing values
#欠損値の補完: fuel、title_status、type、およびstateという列について、欠損値をそれぞれの列の最頻値で補完しています。
df_test['fuel'].fillna(df_test['fuel'].mode()[0], inplace=True)
df_test['title_status'].fillna(df_test['title_status'].mode()[0], inplace=True)
df_test['type'].fillna(df_test['type'].mode()[0], inplace=True)
df_test['state'].fillna(df_test['state'].mode()[0], inplace=True)

# Encoding categorical columns
#カテゴリ変数のエンコード: データフレームのカテゴリ変数（文字列などの非数値型の列）を数値に変換しています。これは、機械学習モデルは数値データしか受け付けないためです。カテゴリ変数の列を選択し、各列をエンコードして数値に変換しています。
categorical_cols = df_test.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_test[col] = df_test[col].astype('category').cat.codes


In [ ]:
import lightgbm as lgb
print(lgb.__version__)

4.0.0


In [ ]:
pip install --upgrade lightgbm


In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error #mean_absolute_error: scikit-learnの平均絶対誤差を計算する関数

# Create LightGBM dataset
train_dataset = lgb.Dataset(X_train, label=y_train) #train_dataset: 訓練データセット
valid_dataset = lgb.Dataset(X_valid, label=y_valid, reference=train_dataset) #valid_dataset: 検証データセット

# Set parameters for LightGBM
params = {
    'objective': 'regression', #objective: タスクの種類（この場合、回帰）
    'metric': 'mape', #metric: 評価指標（この場合、MAPE：平均絶対パーセンテージ誤差）
    'boosting_type': 'gbdt', #boosting_type: ブースティングの種類（この場合、GBDT）
    'num_leaves': 31, #num_leaves: 1つの決定木の最大の葉の数
    'learning_rate': 0.05, #learning_rate: 学習率
    'feature_fraction': 0.9 #feature_fraction: 訓練データの特徴量のサブサンプリング率
}

# Train the model
num_round = 1000 #num_round: 訓練の最大ラウンド数
bst = lgb.train(params, train_dataset, num_round,
                valid_sets=[valid_dataset],  #valid_sets: 検証データセットを指定しています。
                callbacks=[lgb.early_stopping(stopping_rounds=10,  #callbacks: 早期停止のためのコールバック関数を指定しています。ここでは、10ラウンドの間、評価指標が改善されない場合に訓練を停止するように設定しています。
                                verbose=True)] # early_stopping用コールバック関数)
)

# Predict on validation set
y_pred = bst.predict(X_valid, num_iteration=bst.best_iteration) #訓練されたモデルを使用して、検証データセットに対して予測を行っています。

# Calculate MAPE
mape = mean_absolute_error(y_valid, y_pred) / (y_valid.sum() / len(y_valid)) * 100

mape



[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 713
[LightGBM] [Info] Number of data points in the train set: 22025, number of used features: 14
[LightGBM] [Info] Start training from score 13477.637321
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[229]	valid_0's mape: 0.672944


43.45714843054785

In [ ]:
df_train.info()
# df_train["region"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27532 entries, 0 to 27531
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            27532 non-null  int64 
 1   region        27532 non-null  object
 2   year          27532 non-null  int64 
 3   manufacturer  27532 non-null  object
 4   condition     27532 non-null  object
 5   cylinders     27532 non-null  object
 6   fuel          26293 non-null  object
 7   odometer      27532 non-null  int64 
 8   title_status  27076 non-null  object
 9   transmission  27532 non-null  object
 10  drive         27532 non-null  object
 11  size          27532 non-null  object
 12  type          27076 non-null  object
 13  paint_color   27532 non-null  object
 14  state         24228 non-null  object
 15  price         27532 non-null  int64 
dtypes: int64(4), object(12)
memory usage: 3.4+ MB


In [ ]:
data.sort_values(by="year", ascending=False)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
6576,6576,349,2022,10,3,3,2,90464,4,0,1,0,4,5,7,14732
25053,25053,46,2022,27,3,3,2,16945,0,0,1,2,0,0,31,29134
3452,3452,123,2022,7,0,3,2,102290,0,0,0,1,10,1,4,49995
10280,10280,298,2022,11,0,3,2,68142,4,1,1,2,9,1,15,37533
24052,24052,124,2022,3,3,3,2,165074,0,1,1,2,12,1,40,20767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14338,14338,23,1925,4,0,6,2,85,0,1,2,1,3,1,4,17358
21233,21233,46,1925,11,0,6,2,67467,0,0,2,1,10,0,31,17590
19319,19319,246,1922,23,0,5,2,123190,0,0,2,1,3,3,4,31349
5097,5097,278,1922,4,0,5,2,33865,5,0,1,2,9,9,43,2826


In [ ]:
data["state"].unique()

array([ 4, 38, 16, 34,  1, 37, 45, 26, 31, 19, 48, 40, 47,  7, 35, 15,  8,
        9, 32,  3, 36, 23,  5, 33, 49, 42, 22, 39, 14, 43, 44, 12, 10, 20,
       24, 17, 27,  0, 13,  6, 50, 28, 21,  2, 11, 41, 29, 30, 46, 18, 25],
      dtype=int8)

In [ ]:
data.shape

(27532, 16)

In [ ]:
X_valid.head(10)

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
1493,110,2011,27,3,3,2,68568,1,0,0,1,0,1,45
4149,311,2013,7,0,5,2,140563,0,0,1,1,0,0,13
1554,221,2007,36,2,3,2,100309,0,0,1,2,9,5,45
8318,26,2013,7,0,3,2,110866,0,0,1,0,9,4,4
15762,346,1999,7,0,5,2,119261,0,0,2,1,9,9,7
7613,231,2008,22,0,3,2,139384,0,1,1,2,3,4,6
5381,342,2013,11,0,3,2,104369,0,0,0,1,9,0,46
341,15,2012,34,1,3,2,196731,0,0,1,1,9,1,10
12119,250,2008,11,0,3,2,70733,4,1,2,2,3,0,4
16040,189,1995,7,0,6,2,107085,0,1,0,1,8,4,48


In [ ]:
X_valid.shape

(5507, 14)

In [ ]:
X_valid.head()

,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
21617,367,2006,13,0,3,2,156081,0,1,1,0,9,0,38
21326,165,2008,22,1,3,2,143323,0,1,1,0,9,4,18
5317,120,2016,30,0,6,0,130992,0,0,0,1,10,1,4
19378,46,2005,13,1,3,2,131332,1,1,1,0,3,9,31
2540,184,2000,4,0,3,2,126589,0,1,2,2,9,9,4


In [ ]:
df_test.head()

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,27532,western slope,2015,chevrolet,excellent,4 cylinders,gas,92553.0,clean,automatic,fwd,full-size,SUV,red,NaN
1,27533,roseburg,2013,nissan,like new,4 cylinders,gas,134385.0,salvage,automatic,fwd,mid-size,sedan,black,or
2,27534,akron / canton,2011,volkswagen,good,4 cylinders,gas,102489.0,clean,automatic,fwd,full-size,sedan,black,oh
3,27535,denver,2016,jeep,excellent,6 cylinders,diesel,64310.0,clean,automatic,4wd,mid-size,SUV,red,co
4,27536,hickory / lenoir,1999,honda,excellent,8 cylinders,gas,180839.0,rebuilt,automatic,4wd,mid-size,SUV,silver,nc


In [ ]:
df_test.shape

(27537, 15)

In [ ]:
df_test = df_test.drop(columns=['id'])

In [ ]:
df_test.shape

(27537, 14)

In [ ]:
predict = bst.predict(df_test, num_iteration=bst.best_iteration)

In [ ]:
submit[1] = predict

In [ ]:
submit.head()

,0,1
0,27532,12968.996501
1,27533,8641.653550
2,27534,6942.837619
3,27535,29455.038951
4,27536,11557.394890


In [ ]:
submit.to_csv("submission03.csv", index=False, header=None)